# Load the features and initialize the constants

In [3]:
import os
import json
import numpy as np
import pandas as pd
from collections import defaultdict
import matplotlib.pyplot as plt

import tensorflow as tf
from sklearn.model_selection import train_test_split

In [4]:
# ---------------------------------------
# Read feature names
# ---------------------------------------

f = open("../../openface_feature_names.json", "r")
all_features = json.load(f)
f.close()

#Regex to link feature names to what they represent
feature_regex = {
    "action_units": ['AU'],
    "gaze": ['gaze'],
    "shape": ['p_'],
    "landmarks_2d": [' x_', ' y_'],
    "landmarks_3d": [' X_', ' Y_', ' Z_'],
    "eye_lmk_2d": ["lmk_x", "lmk_y"],
    "eye_lmk_3d": ["lmk_X", "lmk_Y", "lmk_Z"],
    "output": ["rating"]
}
features = defaultdict(list)
for feature, regexes in feature_regex.items():
    for col in all_features:
        for regex in regexes:
            if regex in col:
                features[feature].append(col)

#Features to use for training
training_features = ['action_units', 'shape', 'landmarks_2d', 'eye_lmk_2d']
training_features = [ftr_name for feature in training_features for ftr_name in features[feature]]

feature_indices = sorted([all_features.index(feature) for feature in training_features])

FileNotFoundError: [Errno 2] No such file or directory: '../../openface_feature_names.json'

In [ ]:
#Constants
FEATURE_INDICES = feature_indices
MAX_FRAMES_PER_VIDEO = 800 #Pad/Truncate data to maitain 800 frames
FEATURE_COUNT = len(FEATURE_INDICES)
print("Number of features used for training : ", FEATURE_COUNT)

# Deep Learning

In [ ]:
#Generator to load data in batches to avoid memory overflow

#Features and labels can be processed inside this generator
def gen(ftr_files):
    ftr_files = [file.decode() for file in ftr_files]
    for i, file in enumerate(ftr_files):
        #Ignore hidden files such as '.dstore'
        if '.npy' not in file:
            continue
        try: #If any issues with reading the current file or missing label, ignore the video
            ftrs = np.load(f'visual_features/{file}')
            ftrs = ftrs[:, FEATURE_INDICES]
            v_id = file.split('.')[0]
            label = np.load(f'visual_labels/{file}')
        except:
            continue
        
        #Pad 0 valued frames(at the end) if frame count for the video lower than MAX_FRAMES_PER_VIDEO
        #Delete frames(from the front) if frame count grater than MAX_FRAMES_PER_VIDEO
        ftrs = tf.keras.preprocessing.sequence.pad_sequences([ftrs], maxlen=MAX_FRAMES_PER_VIDEO, 
                                                             dtype='float64', padding='post', 
                                                             truncating='pre', value=0.0)
        yield ftrs[0], label

In [ ]:
#Generate an LSTM
def generate_model(input_size, max_length):
    model = tf.keras.Sequential()
    
    model.add(tf.keras.layers.InputLayer(input_shape = (input_size, max_length)))
    
    rnn_layer = tf.keras.layers.LSTM(units = 20, activation = 'sigmoid', dropout = 0, recurrent_dropout = 0, implementation = 1, return_sequences = False)
    model.add(rnn_layer)
    
    dense_layer = tf.keras.layers.Dense(20)
    model.add(dense_layer)
    
    dense_layer = tf.keras.layers.Dense(1)
    model.add(dense_layer)
    
    model.compile(loss = 'mean_squared_error', optimizer = tf.keras.optimizers.Adam(learning_rate = 1e-2))
    model.build()
    model.summary()
    return model

In [ ]:
#Create the train and test datasets
ftr_files = [file for file in os.listdir('visual_features/') if '.npy' in file]
np.random.shuffle(ftr_files)

#Keep x% of data as training
file_count = len(ftr_files)
train_size = 80 #In %age
split_index = (file_count*80)//100

train, test = ftr_files[:split_index], ftr_files[split_index:]

#Training dataset
dataset_train = tf.data.Dataset.from_generator(
     gen, (tf.float64, tf.float64),
     output_shapes=(tf.TensorShape((None, None)), tf.TensorShape(())), args=(train,))
dataset_train_batched = dataset_train.batch(128, drop_remainder=True)
dataset_train_batched_prefetched = dataset_train_batched.prefetch(1)

#Test dataset
dataset_test = tf.data.Dataset.from_generator(
     gen, (tf.float64, tf.float64),
     output_shapes=(tf.TensorShape((None, None)), tf.TensorShape(())), args=(test,))
dataset_test_batched = dataset_test.batch(1)
dataset_test_prefetched = dataset_test_batched.prefetch(1)

In [ ]:
model = generate_model(MAX_FRAMES_PER_VIDEO, FEATURE_COUNT)

In [ ]:
history = model.fit(dataset_train_batched_prefetched, epochs=100, validation_data=dataset_test_batched)#_prefetched)

In [ ]:
plt.plot(history.history['loss'])

In [ ]:
x_test = np.array([ele for batch in dataset_test_batched.as_numpy_iterator() for ele in batch[0]])
y_test = np.array([ele for batch in dataset_test_batched.as_numpy_iterator() for ele in batch[1]])
results = model.predict(x_test)
results = results.reshape(-1)
print(results.shape,y_test.shape)
plt.scatter(results, y_test)

In [ ]:
print(results, y_test)

In [ ]:
tf.keras.losses.MSE(y_test, results)

In [ ]:
tf.keras.utils.plot_model(model)